In [1]:
import lxml

In [19]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["gmailetl"]
mycol = mydb['emails']

In [17]:
from bs4 import BeautifulSoup
import email
import base64
import os.path
import pickle
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle','rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json',SCOPES)
        creds = flow.run_local_server(port = 0)

    with open('token.pickle','wb') as token:
        pickle.dump(creds,token)

service  = build('gmail','v1',credentials=creds)





In [21]:
result = service.users().messages().list(maxResults=10,userId='me').execute()
messages = result.get('messages')
mails = []
index=0
for msg in messages:
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    
    try:
        payload = txt['payload']
        headers = payload['headers']

        for d in headers:
            if d['name'] == 'Subject':
                subject = d['value']
            if d['name'] == 'From':
                sender = d['value']
        
    
        parts = payload.get('parts')[0]
        data = parts['body']['data']
        data = data.replace("-","+").replace("_","/")
        decoded_data = base64.b64decode(data)

        soup = BeautifulSoup(decoded_data,"lxml")
        body = soup.body()
        

        
        body = str(body)
        subject = str(subject)
        sender = str(sender)
        mydict = {"_id" : index,  "Sender": sender, "Subject": subject, "Body" : body }
        x = mycol.insert_one(mydict)
        index = index+1
    except:
        pass
    

TypeError: documents must be a non-empty list